In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.stats import norm

In [2]:
class Buyer:
  def __init__(self, mu, sigma):
    """
    Buyer using a multivariate normal distribution for valuations.
    :param mu: Mean vector (list or 1D numpy array)
    :param sigma: Covariance matrix (2D numpy array)
    """
    self.mu = np.array(mu)
    self.sigma = np.array(sigma)
    self.dimension = self.mu.shape[0]

    # Validate that the covariance matrix is square and matches the mean vector
    if self.sigma.shape != (self.dimension, self.dimension):
      raise ValueError("Covariance matrix shape must match (len(mu), len(mu))")

  def round(self, prices):
    """
    Simulates a round where the buyer evaluates multiple prices.
    :param prices: List or array of offered prices (must match dimension)
    :return: Binary array indicating purchases (1 = buy, 0 = no buy)
    """
    valuations = np.random.multivariate_normal(self.mu, self.sigma)
    prices = np.array(prices)

    if len(prices) != self.dimension:
      raise ValueError("Length of prices must match length of mean vector")

    return (valuations > prices).astype(int)

In [3]:
mu = [1.0, 0.8, 1.2]
sigma = [
    [0.1, 0.05, 0.02],
    [0.05, 0.2, 0.03],
    [0.02, 0.03, 0.15]
]

buyer = Buyer(mu, sigma)
prices = [0.9, 0.7, 1.1]
response = buyer.round(prices)
print("Buy decisions:", response)

Buy decisions: [1 1 1]


In [4]:
def compute_best_price(prices, rho, buyer, num_samples=10000):
  """
  Compute optimal randomized pricing strategy using vectorized buyer simulations.

  Parameters:
  - prices: list of possible prices
  - rho: maximum expected units sold (capacity constraint)
  - buyer: Buyer object (with multivariate normal valuation)
  - num_samples: number of samples to estimate win probabilities

  Returns:
  - gamma: optimal price distribution
  - expected_revenue: expected revenue under strategy
  - expected_units_sold: expected units sold under strategy
  """
  prices = np.array(prices)
  num_prices = len(prices)

  # Generate all valuations at once using vectorized sampling
  valuations = np.random.multivariate_normal(buyer.mu, buyer.sigma, size=num_samples)

  # Compute win probabilities: fraction of times valuation exceeds price
  win_probabilities = np.mean(valuations > prices, axis=0)

  # Linear programming setup
  c = -(prices * win_probabilities)  # Negate for minimization

  A_ub = [win_probabilities]  # Expected units sold <= rho
  b_ub = [rho]

  A_eq = [np.ones(num_prices)]  # gamma is a probability distribution
  b_eq = [1]

  bounds = [(0, 1)] * num_prices

  res = optimize.linprog(c, A_ub=A_ub, b_ub=b_ub,
                          A_eq=A_eq, b_eq=b_eq,
                          bounds=bounds, method='highs')

  gamma = res.x
  expected_revenue = -res.fun
  expected_units_sold = np.sum(gamma * win_probabilities)

  return gamma, expected_revenue, expected_units_sold

In [5]:
mu = [1.0, 0.8, 0.2]
sigma = [
    [0.1, 0.05, 0.02],
    [0.05, 0.2, 0.03],
    [0.02, 0.03, 0.15]
]

buyer = Buyer(mu, sigma)
prices = np.linspace(0, 1, 3)
rho = 0.5  # Maximum expected units sold

gamma, revenue, units = compute_best_price(prices, rho, buyer)

print("Optimal price distribution (gamma):", gamma)
print("Expected revenue:", revenue)
print("Expected units sold:", units)

Optimal price distribution (gamma): [0.         0.65747221 0.34252779]
Expected revenue: 0.2535794153972828
Expected units sold: 0.49999999999999994
